usunac atrybut klasy przed klastrowaniem
efficiency of algo, ile razy przy deterministycznej inicjalizacji trzeba bylo odpalic, itp

In [3]:
import numpy as np

In [44]:
X = np.array([[1.0, 1.0], [1.0, 5.0], [3.0, 2.0]])
cluster = [1, 2]
ccc = np.array([0.0, 0.0])

In [42]:
def center(X, cluster, ccc):
    nn, mm = X.shape
    for jj in range(mm):
        zz = X[:, jj]
        zc = zz[cluster]
        ccc[jj] = np.mean(zc, dtype=np.float32)
    
    return ccc

print(center(X, cluster, ccc))

[2.  3.5]


In [45]:
def dist(X, remains, range_in, a):
    nn, mm = X.shape
    rr = len(remains)
    distan = []
    
    for jj in range(mm):
        z = X[:, jj]
        zz = z[remains]
        y = zz - a[jj]
        y = y / range_in[jj]
        yy = np.dot(y, y.T)
        distan.append(yy)
        
    return np.array(distan)

remains = [*range(X.shape[0])]
range_in = []
me = []

for j in range(X.shape[1]):
    range_in.append(max(X[:, j]) - min(X[:, j]))
    me.append(np.mean(X[:, j]))

print(dist(X, remains, range_in, me))

[0.66666667 0.54166667]
